<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [ ]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [ ]:
# 1.a) Consolidación y normalización de columnas

# leer ambos CSV
dfs = [pd.read_csv(url) for url in archivos_anio]

# normalizar los nombres de columnas a minúscula (y sin espacios)
dfs = [df.rename(columns=lambda c: c.strip().lower()) for df in dfs]

# concatenar por filas para consolidar en un solo DataFrame
df_anio = pd.concat(dfs, ignore_index=True, sort=False)


# 1.b) Limpieza de df_codigos

# Normalizar nombres de columnas
df_codigos.columns = df_codigos.columns.str.strip().str.lower()

# Identificar códigos ISO asociados a >1 país
dup_table = (df_codigos.groupby('codigo_iso', as_index=False)
             .agg(paises_distintos=('pais','nunique')))
conflict_codes = dup_table.query('paises_distintos > 1')['codigo_iso'].tolist()
print("Códigos en conflicto:", conflict_codes)

# Explorar las filas en conflicto (evidencia)
display(df_codigos[df_codigos['codigo_iso'].isin(conflict_codes)]
        .sort_values(['codigo_iso','pais']))

# Eliminar el registro incorrecto (en este dataset: 'malo')
df_codigos = df_codigos[df_codigos['pais'].str.lower().ne('malo')].copy()

# Verificación: ya no deben quedar códigos con >1 país
resto = (df_codigos.groupby('codigo_iso')['pais'].nunique() > 1).sum()
print("Conflictos restantes (debería ser 0):", resto)


# 1.c) Unión por 'codigo_iso' conservando solo coincidencias

# asegurar que en df_codigos cada codigo_iso tenga un único país
assert not df_codigos['codigo_iso'].duplicated().any(), "df_codigos tiene codigo_iso repetidos"

# combinar df_anio con df_codigos usando 'codigo_iso' y conservando solo coincidencias
df = pd.merge(
    df_anio,
    df_codigos[['codigo_iso', 'pais']],  # columnas necesarias desde df_codigos
    on='codigo_iso',
    how='inner',                         # <-- unión interna: solo registros con coincidencia en ambas fuentes
    validate='many_to_one'               # cada codigo_iso en df_codigos -> un solo país
).copy()

# reordenar columnas para consistencia
df = df[['codigo_iso', 'pais', 'anio', 'indice', 'ranking']]

# verificación rápida
print("df shape:", df.shape)
df.head()




Códigos en conflicto: ['ZWE']


,codigo_iso,pais
179,ZWE,Zimbabue
180,ZWE,malo


Conflictos restantes (debería ser 0): 0
df shape: (3060, 5)


,codigo_iso,pais,anio,indice,ranking
0,AFG,Afghanistán,2001,35.5,59.0
1,AGO,Angola,2001,30.2,50.0
2,ALB,Albania,2001,NaN,NaN
3,AND,Andorra,2001,NaN,NaN
4,ARE,Emiratos Árabes Unidos,2001,NaN,NaN




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [ ]:
# 2) Estructura del DataFrame

# ¿Cuántas filas y columnas?
n_filas, n_columnas = df.shape
print(f"Filas: {n_filas}")
print(f"Columnas: {n_columnas}")

# ¿Cuáles son los nombres de las columnas?
cols = df.columns.tolist()
print("Nombres de columnas:", cols)

# ¿Qué tipo de datos tiene cada columna?
tipos = df.dtypes.to_frame("dtype")
display(tipos)

# ¿Hay columnas con un tipo de dato inesperado?
#     Heurística: en columnas 'object' detectamos si parecen fechas o números guardados como texto.
import numpy as np
import pandas as pd

obj_cols = [c for c in df.columns if df[c].dtype == "object"]
sospechosas = []

for c in obj_cols:
    serie = df[c].dropna().astype(str).str.strip()
    # a) ¿parecen fechas (ej: "2001-05-01")?
    parsed_date = pd.to_datetime(serie, errors="coerce", format=None)
    frac_fechas = parsed_date.notna().mean() if len(serie) else 0.0

    # b) ¿parecen números guardados como texto (ej: "12.3")?
    parsed_num = pd.to_numeric(serie.str.replace(",", "."), errors="coerce")
    frac_numeros = parsed_num.notna().mean() if len(serie) else 0.0

    if frac_fechas > 0.8:
        sospechosas.append((c, "posible fecha en texto", frac_fechas))
    elif frac_numeros > 0.8:
        sospechosas.append((c, "posible número en texto", frac_numeros))

print("\nColumnas con tipo potencialmente inesperado:")
if sospechosas:
    for c, motivo, frac in sospechosas:
        print(f" - {c}: {motivo} (≈{frac:.0%} de las filas)")
else:
    print(" - No se detectaron columnas sospechosas en las columnas tipo 'object'.")


Filas: 3060
Columnas: 5
Nombres de columnas: ['codigo_iso', 'pais', 'anio', 'indice', 'ranking']


,dtype
codigo_iso,object
pais,object
anio,int64
indice,float64
ranking,float64



Columnas con tipo potencialmente inesperado:
 - No se detectaron columnas sospechosas en las columnas tipo 'object'.


/tmp/ipython-input-3324776500.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_date = pd.to_datetime(serie, errors="coerce", format=None)
/tmp/ipython-input-3324776500.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_date = pd.to_datetime(serie, errors="coerce", format=None)


Filas (observaciones): 3060

Columnas: 5

Nombres de columnas: codigo_iso, pais, anio, indice, ranking

Tipos por columna:

codigo_iso → object (texto)

pais → object (texto)

anio → int64 (entero)

indice → float64 (numérico)

ranking → float64 (numérico)

¿Tipos inesperados (p. ej., fechas como strings)?
No. Se revisaron las columnas tipo object y no se detectaron patrones de fecha ni números guardados como texto; anio está correctamente en entero.

In [ ]:
# 2) Resumen estadístico

# 1) describe(): resumen de 'indice' y 'ranking'
print("Resumen estadístico de 'indice' y 'ranking'")
desc = df[['indice', 'ranking']].describe()
display(desc)

# 2) min, max y promedio (mean) de 'indice'
ind_stats = df['indice'].agg(min='min', max='max', mean='mean')
print("\nIndice — min/max/mean:")
print(ind_stats)

# 3) países con valores extremos en indice y ranking (incluye empates)
i_min_idx = df['indice'].idxmin(skipna=True)
i_max_idx = df['indice'].idxmax(skipna=True)
r_min_idx = df['ranking'].idxmin(skipna=True)
r_max_idx = df['ranking'].idxmax(skipna=True)

i_min = df.loc[df['indice'].eq(df.loc[i_min_idx, 'indice']),
               ['pais', 'anio', 'indice']].sort_values(['indice','anio','pais'])
i_max = df.loc[df['indice'].eq(df.loc[i_max_idx, 'indice']),
               ['pais', 'anio', 'indice']].sort_values(['indice','anio','pais'])
r_min = df.loc[df['ranking'].eq(df.loc[r_min_idx, 'ranking']),
               ['pais', 'anio', 'ranking']].sort_values(['ranking','anio','pais'])
r_max = df.loc[df['ranking'].eq(df.loc[r_max_idx, 'ranking']),
               ['pais', 'anio', 'ranking']].sort_values(['ranking','anio','pais'])

print("\nPaís(es) con INDICE mínimo (mayor libertad):")
display(i_min)

print("País(es) con INDICE máximo (menor libertad):")
display(i_max)

print("País(es) con RANKING mínimo (mejor posición):")
display(r_min)

print("País(es) con RANKING máximo (peor posición):")
display(r_max)



Resumen estadístico de 'indice' y 'ranking'


,indice,ranking
count,2664.000000,2837.000000
mean,205.782316,477.930913
std,2695.525264,6474.935347
min,0.000000,1.000000
25%,15.295000,34.000000
50%,28.000000,70.000000
75%,41.227500,110.000000
max,64536.000000,121056.000000



Indice — min/max/mean:
min         0.000000
max     64536.000000
mean      205.782316
Name: indice, dtype: float64

País(es) con INDICE mínimo (mayor libertad):


,pais,anio,indice
1304,Dinamarca,2008,0.0
1313,Finlandia,2008,0.0
1335,Irlanda,2008,0.0
1382,Noruega,2008,0.0
1412,Suecia,2008,0.0
1493,Finlandia,2009,0.0
1518,Islandia,2009,0.0
1562,Noruega,2009,0.0
1561,Países Bajos,2009,0.0
1592,Suecia,2009,0.0


País(es) con INDICE máximo (menor libertad):


,pais,anio,indice
2069,Kosovo,2014,64536.0


País(es) con RANKING mínimo (mejor posición):


,pais,anio,ranking
53,Finlandia,2001,1.0
78,Islandia,2001,1.0
122,Noruega,2001,1.0
121,Países Bajos,2001,1.0
233,Finlandia,2002,1.0
258,Islandia,2002,1.0
302,Noruega,2002,1.0
301,Países Bajos,2002,1.0
404,Dinamarca,2003,1.0
510,Eslovaquia,2003,1.0


País(es) con RANKING máximo (peor posición):


,pais,anio,ranking
2249,Kosovo,2015,121056.0


Resumen con .describe() (índice y ranking):

indice: count = 2664 (hay faltantes), mediana ≈ 28, IQR ≈ 15.30–41.23. La media ≈ 205.78 es mucho mayor que la mediana → hay valores atipicos muy altos.

ranking: count = 2837 (hay faltantes), mediana = 70, IQR ≈ 34–110. La media ≈ 477.93 también está inflada por valores extremos.

Nota: en este dataset, menor indice = mayor libertad y menor ranking = mejor posición.

indice — mínimo, máximo y promedio:

mínimo: 0.00

máximo: 64 536.00 (claramente atípico)

promedio: 205.78 (distorsionado por valores atipicos)

Países con valores extremos:

indice mínimo (0.0, mayor libertad): aparecen varios países europeos (2008–2009), entre ellos Dinamarca, Finlandia, Irlanda, Noruega, Suecia, Países Bajos, Islandia, Suiza.

indice máximo (64 536.0, menor libertad): Kosovo (2014) → valor anomalo.

ranking mínimo (1.0, mejor posición): múltiples años para Finlandia, Islandia, Noruega, Países Bajos, Dinamarca, Suiza, Irlanda, Luxemburgo, Estonia, Austria, Nueva Zelanda, Eslovaquia, entre otros.

ranking máximo (121 056.0, peor posición): Kosovo (2015) → también anomalo.

In [ ]:
# 2) Datos faltantes

# ¿Cuántos valores nulos hay en cada columna?
na_counts = df.isna().sum()

# ¿Qué proporción (porcentaje) de nulos hay en cada columna?
na_pct = (df.isna().mean() * 100).round(2)

# Tabla resumen ordenada por mayor % de faltantes
na_summary = (
    pd.DataFrame({'nulos': na_counts, 'porc_%': na_pct})
    .sort_values('porc_%', ascending=False)
)
print("Nulos por columna:")
display(na_summary)

# ¿Qué proporción de observaciones (filas) tienen al menos un valor faltante?
rows_any_na = df.isna().any(axis=1).sum()
rows_any_na_pct = round(rows_any_na / len(df) * 100, 2)
print(f"Filas con ≥1 valor faltante: {rows_any_na} de {len(df)} ({rows_any_na_pct}%)")

# ¿Hay columnas con más del 30% de datos faltantes?
cols_gt30 = na_summary[na_summary['porc_%'] > 30]
if cols_gt30.empty:
    print("Columnas con >30% de faltantes: ninguna")
else:
    print("Columnas con >30% de faltantes:")
    display(cols_gt30)


Nulos por columna:


,nulos,porc_%
indice,396,12.94
ranking,223,7.29
codigo_iso,0,0.00
anio,0,0.00
pais,0,0.00


Filas con ≥1 valor faltante: 397 de 3060 (12.97%)
Columnas con >30% de faltantes: ninguna


Valores nulos por columna:

indice: 396 nulos (12.94%)

ranking: 223 nulos (7.29%)

codigo_iso: 0%

anio: 0%

pais: 0%

Proporción de observaciones con ≥1 valor faltante:
397 de 3060 → 12.97% de las filas.

Columnas con más del 30% de faltantes:
Ninguna.

In [ ]:
# 2) Unicidad y duplicados

# ¿Cuántos países distintos (pais)?
n_paises = df['pais'].nunique(dropna=True)
print("Países distintos:", n_paises)

# ¿Cuántos años distintos (anio)?
n_anios = df['anio'].nunique(dropna=True)
print("Años distintos:", n_anios)

# ¿Existen filas duplicadas EXACTAS (todas las columnas)? ¿Cuántas?
dup_mask = df.duplicated(keep='first')   # True en las repeticiones exactas
n_dups = dup_mask.sum()
print("Filas duplicadas (exactamente iguales):", n_dups)



Países distintos: 179
Años distintos: 17
Filas duplicadas (exactamente iguales): 0


Países distintos (pais): 179

Años distintos (anio): 17

Filas duplicadas (exactamente iguales): 0

In [ ]:
# Validación cruzada de columnas

# Normalizar si acaso (evita falsos positivos por mayúsculas/espacios)
df_chk = df.assign(
    codigo_iso=df['codigo_iso'].astype(str).str.strip().str.upper(),
    pais=df['pais'].astype(str).str.strip()
)

# ¿Hay nulos en las columnas clave?
print("Nulos en columnas clave:")
print(df_chk[['codigo_iso', 'pais']].isnull().sum(), "\n")

# ¿Un mismo ISO está asociado a >1 país?  (lo que pide el enunciado)
iso_to_pais = df_chk.groupby('codigo_iso')['pais'].nunique().sort_values(ascending=False)
conf_iso = iso_to_pais[iso_to_pais > 1]
print("Códigos ISO asociados a >1 país (inconsistencias):")
display(conf_iso)

# (si existieran, mostramos ejemplos)
if not conf_iso.empty:
    display(
        df_chk[df_chk['codigo_iso'].isin(conf_iso.index)]
        .sort_values(['codigo_iso','pais','anio'])
        .head(20)
    )


Nulos en columnas clave:
codigo_iso    0
pais          0
dtype: int64 

Códigos ISO asociados a >1 país (inconsistencias):


,pais
codigo_iso,


Nulos en columnas clave:
codigo_iso: 0 — pais: 0.

¿Un mismo codigo_iso asociado a >1 pais?
No. El listado de inconsistencias quedó vacío tras la limpieza (1.b).

Comentario: El mapeo codigo_iso → pais es consistente. Es normal que el mismo par (codigo_iso, pais) se repita en distintos años, y eso no es una inconsistencia.




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [ ]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

# L1) Filtrar el DataFrame final a la lista de países de América (por codigo_iso)
df_america = df[df['codigo_iso'].isin(america)].copy()

# a) Con ciclo for
# Para cada año: país con menor índice (mejor libertad) y mayor índice (peor libertad)
years = sorted(df_america['anio'].unique())
res_for = []
for y in years:                                            # recorrer años
    g = df_america.loc[(df_america['anio']==y) &           # subset por año + descartar NaN en indice
                       (df_america['indice'].notna()),
                       ['anio','pais','codigo_iso','indice']]
    if g.empty:
        continue
    i_min = g.loc[g['indice'].idxmin()]                    # país con menor indice
    i_max = g.loc[g['indice'].idxmax()]                    # país con mayor indice
    res_for.append({
        'anio': y,
        'pais_indice_min': i_min['pais'], 'iso_min': i_min['codigo_iso'], 'indice_min': i_min['indice'],
        'pais_indice_max': i_max['pais'], 'iso_max': i_max['codigo_iso'], 'indice_max': i_max['indice'],
    })
res_for = pd.DataFrame(res_for).sort_values('anio').reset_index(drop=True)

# b) Vectorizado con groupby (sin for explícito)
g = df_america[df_america['indice'].notna()].copy()        # descartar NaN en indice
idx_min = g.groupby('anio')['indice'].idxmin()             # índice de la fila con menor indice por año
idx_max = g.groupby('anio')['indice'].idxmax()             # índice de la fila con mayor indice por año

min_df = g.loc[idx_min, ['anio','pais','codigo_iso','indice']]\
          .rename(columns={'pais':'pais_indice_min','codigo_iso':'iso_min','indice':'indice_min'})
max_df = g.loc[idx_max, ['anio','pais','codigo_iso','indice']]\
          .rename(columns={'pais':'pais_indice_max','codigo_iso':'iso_max','indice':'indice_max'})

res_groupby = (min_df.merge(max_df, on='anio', how='outer')  # tabla final por año (min+max)
                      .sort_values('anio')
                      .reset_index(drop=True))
display(res_for.head())
display(res_groupby.head())



,anio,pais_indice_min,iso_min,indice_min,pais_indice_max,iso_max,indice_max
0,2001,Canadá,CAN,0.8,Cuba,CUB,90.30
1,2002,Trinidad y Tobago,TTO,1.0,Cuba,CUB,97.83
2,2003,Trinidad y Tobago,TTO,2.0,Argentina,ARG,35826.00
3,2004,Trinidad y Tobago,TTO,2.0,Cuba,CUB,87.00
4,2005,Bolivia,BOL,4.5,Cuba,CUB,95.00


,anio,pais_indice_min,iso_min,indice_min,pais_indice_max,iso_max,indice_max
0,2001,Canadá,CAN,0.8,Cuba,CUB,90.30
1,2002,Trinidad y Tobago,TTO,1.0,Cuba,CUB,97.83
2,2003,Trinidad y Tobago,TTO,2.0,Argentina,ARG,35826.00
3,2004,Trinidad y Tobago,TTO,2.0,Cuba,CUB,87.00
4,2005,Bolivia,BOL,4.5,Cuba,CUB,95.00


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [ ]:
# 4) Análisis anual del índice por país

# preparar las columnas necesarias (y opcionalmente descartar NaN en indice)
base = df[['pais', 'anio', 'indice']].copy()

# construir la tabla dinámica: filas = país, columnas = año, valores = índice MÁXIMO
pivot_indice_max = pd.pivot_table(
    base,
    index='pais',
    columns='anio',
    values='indice',
    aggfunc='max',      # índice máximo por país y año
    fill_value=0        # reemplazar nulos resultantes con 0
)

# ordenar para una visualización consistente
pivot_indice_max = pivot_indice_max.sort_index(axis=0).sort_index(axis=1)
display(pivot_indice_max)

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,81.3,89.17,86.88,73.25,67.25,79.25,86.17,81.67,75.75,71.78,72.36,72.63,74.27,73.96,75.05,74.93
West Bank y Gaza,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42.90,42.96,44.68
Yemen,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,65.80,62.23,61.66


In [ ]:
# 4) Preguntas adicionales con la pivot
piv = pivot_indice_max

# a) País con el mayor valor de 'indice' en toda la tabla y el menor (≠ 0)
cells = piv.stack()

# máximo global y sus ubicaciones (podrían ser varias celdas)
val_max = cells.max()
max_cells = cells[cells == val_max].sort_index()   # (pais, anio) -> valor
# mínimo global distinto de cero y sus ubicaciones
val_min_nz = cells[cells > 0].min()
min_cells = cells[cells == val_min_nz].sort_index()

print("a) Máximo global:")
display(max_cells.to_frame("indice_max"))
print("a) Mínimo global (≠0):")
display(min_cells.to_frame("indice_min"))

# b) Años con promedio más alto y más bajo (promedio sobre países)
mean_year = piv.mean(axis=0)
yr_max_mean = mean_year.idxmax()
yr_min_mean = mean_year.idxmin()
print("b) Promedio por año (muestra):")
display(mean_year.to_frame("promedio").sort_index())
print(f"Año con promedio MÁS alto: {yr_max_mean}  | valor: {mean_year[yr_max_mean]:.2f}")
print(f"Año con promedio MÁS bajo: {yr_min_mean}  | valor: {mean_year[yr_min_mean]:.2f}")

# c) País con mayor variabilidad (max - min a lo largo del tiempo)
var_country = piv.max(axis=1) - piv.min(axis=1)
country_var_max = var_country.idxmax()
print("c) País con mayor variabilidad:")
display(var_country.sort_values(ascending=False).to_frame("variabilidad").head(10))
print(f"Mayor variabilidad: {country_var_max} → {var_country.max():.2f}")

# d) Países con índice constante a lo largo de todos los años
# constante si el número de valores distintos por fila es 1
const_mask = piv.nunique(axis=1) == 1
const_countries = piv.index[const_mask].tolist()
print("d) Países con índice constante en todos los años registrados:")
print(const_countries if const_countries else "Ninguno")

# e) Países sin ningún dato (todas las celdas 0 en la pivot)
# suma por país igual a 0 → todos ceros
no_data = piv.sum(axis=1) == 0
no_data_countries = piv.index[no_data].tolist()
print("e) Países sin ningún dato (todo 0 en la pivot):")
print(no_data_countries if no_data_countries else "Ninguno")


a) Máximo global:


,,indice_max
pais,anio,
Kosovo,2014,64536.0


a) Mínimo global (≠0):


indice_min
pais         anio            
Austria      2009         0.5
Dinamarca    2003         0.5
             2004         0.5
Eslovaquia   2003         0.5
Estonia      2008         0.5
Finlandia    2001         0.5
             2002         0.5
             2003         0.5
             2004         0.5
             2005         0.5
Irlanda      2003         0.5
             2004         0.5
             2005         0.5
Islandia     2001         0.5
             2002         0.5
             2003         0.5
             2004         0.5
             2005         0.5
Noruega      2001         0.5
             2002         0.5
             2003         0.5
             2004         0.5
Países Bajos 2001         0.5
             2002         0.5
             2003         0.5
             2004         0.5
             2005         0.5
Suiza        2003         0.5
             2004         0.5

b) Promedio por año (muestra):


,promedio
anio,
2001,20.032402
2002,22.843520
2003,225.209777
2004,229.539218
2005,133.833128
2006,143.132346
2007,126.574972
2008,137.877709
2009,233.884190


Año con promedio MÁS alto: 2013  | valor: 449.11
Año con promedio MÁS bajo: 2001  | valor: 20.03
c) País con mayor variabilidad:


,variabilidad
pais,
Kosovo,64536.00
Tonga,37126.00
Senegal,37110.00
Argentina,35814.67
Corea del Norte,104.75
Laos,94.83
Somalía,77.50
Bhutan,75.30
Maldivas,69.17


Mayor variabilidad: Kosovo → 64536.00
d) Países con índice constante en todos los años registrados:
Ninguno
e) Países sin ningún dato (todo 0 en la pivot):
Ninguno


a) País con el mayor y el menor (≠0) valor de indice

Máximo global de indice (menor libertad): Kosovo (2014) con 64 536.0 → valor claramente atípico.

Mínimo global de indice (≠0, mayor libertad): 0.5 en varios países y años, entre ellos
Finlandia, Noruega, Islandia, Países Bajos, Dinamarca, Suiza, Irlanda, Austria, Estonia, Eslovaquia (años como 2001–2005 y 2008–2009), entre otros.

b) Años con promedios de indice más altos y más bajos

Promedio MÁS alto: 2013, ≈ 449.11.

Promedio MÁS bajo: 2001, ≈ 20.03.

Nota: estos promedios se calcularon sobre la tabla dinámica con fill_value=0, por lo que las combinaciones país-año sin dato cuentan como 0.

c) País con mayor variabilidad (máx − mín a lo largo del tiempo)

Mayor variabilidad: Kosovo, ≈ 64 536.00.
(Otros altos: Tonga ≈ 37 126, Senegal ≈ 37 110, Argentina ≈ 35 814.67 — todos muy elevados/atípicos.)

d) Países con indice constante en todos los años registrados

Ninguno.

e) Países sin ningún dato (toda la fila en 0 en la pivot)

Ninguno

¿Por qué no hay países sin ningún dato (toda la fila en 0)?

Porque en el dataset cada país tiene al menos un indice no nulo en algún año. En la pivot_table:

Si un país tuviera filas pero todos sus indice fueran NaN, al calcular el max quedaría NaN y fill_value=0 lo convertiría en 0 en todas las columnas → aparecería como “todo 0”.

Si un país no tuviera ninguna fila en el DataFrame original, ni siquiera aparecería en la tabla dinámica.